In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles = []
title_ids = []
nums = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
topics = pd.DataFrame()
topics['qid'] = nums
topics['query'] = titles

qrels_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

docnos = qrels.docno

In [5]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print(subdir)
            path = subdir + '/'+ file
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dph = pt.BatchRetrieve(index, wmodel="DPH")
            dph_rm3 = (pt.BatchRetrieve(indexref, wmodel="DPH") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DPH"))
            bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="BM25"))
            tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
            bm25 = pt.BatchRetrieve(index, wmodel="BM25")
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM")
            dlm_rm3 = (pt.BatchRetrieve(indexref, wmodel="DirichletLM") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DirichletLM"))
            models = [dph, dph_rm3, tf_idf, bm25_rm3, bm25, dlm, dlm_rm3]
            print(pt.Experiment(models, topics, qrels, eval_metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"]))
            

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
                                                name  ndcg_cut_5     bpref  \
0                                            BR(DPH)    0.315636  0.644521   
1  Compose(Compose(BR(DPH), <pyterrier.rewrite.RM...    0.321356  0.667382   
2                                         BR(TF_IDF)    0.486396  0.724771   
3  Compose(Compose(BR(BM25), <pyterrier.rewrite.R...    0.481536  0.749372   
4                                           BR(BM25)    0.482386  0.725052   
5                                    BR(DirichletLM)    0.605145  0.726433   
6  Compose(Compose(BR(DirichletLM), <pyterrier.re...    0.525739  0.702217   

   ndcg_cut_25  
0     0.306881  
1     0.316702  
2     0.446201  
3     0.448463  
4     0.445090  
5     0.524329  
6     0.387855  
/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
               

In [4]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print('\n' + subdir)
            path = subdir + '/'+ file
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dph = pt.BatchRetrieve(index, wmodel="DPH")
            dph_rm3 = (pt.BatchRetrieve(indexref, wmodel="DPH") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DPH"))
            bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="BM25"))
            tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
            bm25 = pt.BatchRetrieve(index, wmodel="BM25")
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM")
            dlm_rm3 = (pt.BatchRetrieve(indexref, wmodel="DirichletLM") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DirichletLM"))
            models = [dph, dph_rm3, tf_idf, bm25_rm3, bm25, dlm, dlm_rm3]
        
            for model in models:
                res= model.transform(topics)
                res_copy = res[res['docno'].isin(docnos)].copy()
                print(f'model : {model}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])}')
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
model : BR(DPH), res: {'ndcg_cut_5': 0.6195559003690524, 'bpref': 0.6445214979422608, 'ndcg_cut_25': 0.6412295985168942}
model : Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7fe6dd3ed190>), BR(DPH)), res: {'ndcg_cut_5': 0.6400572193920301, 'bpref': 0.667382205137545, 'ndcg_cut_25': 0.6544399199256083}
model : BR(TF_IDF), res: {'ndcg_cut_5': 0.6283797590144036, 'bpref': 0.7247708473797921, 'ndcg_cut_25': 0.6753707122077693}
model : Compose(Compose(BR(BM25), <pyterrier.rewrite.RM3 object at 0x7fe6dd3ed050>), BR(BM25)), res: {'ndcg_cut_5': 0.6704911407713139, 'bpref': 0.7493722124385017, 'ndcg_cut_25': 0.7055810321214193}
model : BR(BM25), res: {'ndcg_cut_5': 0.629094689978407, 'bpref': 0.7250523093453518, 'ndcg_cut_25': 0.6765302738349726}
model : BR(DirichletLM), res: {'ndcg_cut_5': 0.7111994374419488, 'bpref': 0.7264330830118617, 'ndcg_cut_25': 0.7229021475536